In [1]:
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt

In [17]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for i in tqdm(filelist[-400:]):
    df0 = pd.read_csv(os.path.join(path, i))
    df = pd.concat([df, df0])

100%|██████████| 400/400 [00:02<00:00, 162.00it/s]


In [18]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()

In [19]:
def caculate_adx(df, length=14):
    """

    :param df:传入的参数
    :param length: 时间周期长度
    :return: df['g1.ADX']
    """
    df = df.copy()
    # 计算真实波幅(TR)
    df['H-L'] = df['h'] - df['l']
    df['H-PreClose'] = np.abs(df['h'] - df['c'].shift(1))
    df['L-PreClose'] = np.abs(df['l'] - df['c'].shift(1))
    df['TR'] = df[['H-L', 'H-PreClose', 'L-PreClose']].max(axis=1)

    # 计算+DM和-DM
    df['H-PreH'] = df['h'] - df['h'].shift(1)
    df['L-PreL'] = df['l'].shift(1) - df['l']
    df['+DM'] = np.where((df['H-PreH'] > df['L-PreL']) & (df['H-PreH'] > 0), df['H-PreH'], 0)
    df['-DM'] = np.where((df['L-PreL'] > df['H-PreH']) & (df['L-PreL'] > 0), df['L-PreL'], 0)

    # 计算TR、+DM、-DM的N日移动平均
    # 使用Wilder的平滑方法
    df['TR_SMA'] = df['TR'].rolling(window=length).mean()
    df['+DM_SMA'] = df['+DM'].rolling(window=length).mean()
    df['-DM_SMA'] = df['-DM'].rolling(window=length).mean()

    # for i in range(length, len(df)):
    #     df.loc[df.index[i], 'TR_SMA'] = (df.loc[df.index[i - 1], 'TR_SMA'] * (length - 1) + df.loc[
    #         df.index[i], 'TR']) / length
    #     df.loc[df.index[i], '+DM_SMA'] = (df.loc[df.index[i - 1], '+DM_SMA'] * (length - 1) + df.loc[
    #         df.index[i], '+DM']) / length
    #     df.loc[df.index[i], '-DM_SMA'] = (df.loc[df.index[i - 1], '-DM_SMA'] * (length - 1) + df.loc[
    #         df.index[i], '-DM']) / length
    # 计算+DI和-DI
    df['+DI'] = (df['+DM_SMA'] / df['TR_SMA']) * 100
    df['-DI'] = (df['-DM_SMA'] / df['TR_SMA']) * 100

    # 计算DX
    df['DX'] = (np.abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100

    # 计算ADX (DX的N日移动平均)
    # df['ADX'] = df['DX'].rolling(window=length).mean()
    df['ADX'] = 0

    df = df.reset_index(drop=True)  # 在关键步骤重置索引
    for i in range(length - 1, len(df)):
        df.loc[df.index[i], 'ADX'] = (df.loc[df.index[i - 1], 'ADX'] * (length - 1) + df.loc[
            df.index[i], 'DX']) / length

    return df['ADX']

In [20]:
df1 = pd.read_csv('./ADXH.csv')
length = 14  # 默认交易周期为14
df1['g1.ADX'] = 0
df1['g1.ADX'] = df1['g1.ADX'].astype(np.float64)
df1['g1.ADX'] = caculate_adx(df=df1, length=length)

C:\Users\yyx\AppData\Local\Temp\ipykernel_12284\433047880.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.8415633863009258' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df.index[i], 'ADX'] = (df.loc[df.index[i - 1], 'ADX'] * (length - 1) + df.loc[


In [24]:
df1 = df1[length-1:]  # 去除空置
df1

,t,o,h,l,c,v,g1.ADX
13,1662433200000,19806.0,19815.9,19694.2,19780.2,16485.985,1.841563
14,1662436800000,19780.3,19839.9,19744.0,19754.1,17001.593,3.719128
15,1662440400000,19754.1,19829.3,19724.0,19772.2,16109.126,4.820922
16,1662444000000,19772.3,19880.1,19755.0,19848.2,18069.820,6.368098
17,1662447600000,19848.3,19940.0,19836.0,19900.7,28810.795,8.596788
...,...,...,...,...,...,...,...
9581,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,48.780720
9582,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,48.772804
9583,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,48.603426
9584,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,47.934711


In [25]:
df1.to_csv('./ADXH.csv', index=False)